# Bibliotecas

In [94]:
import pandas as pd     #Manipulação dos dados
import numpy as np      #Da suporte a arrays e interações matemáticas com eles

import plotly.express as px                 #Criação de gráficos interativos
import matplotlib.pyplot as plt             #Vizualização de dados
import plotly.graph_objects as go           #Fornece algumas funções de interatividade com os gráficos
from plotly.subplots import make_subplots   #Permite alocar vários gráficos em uma única figura

from sklearn.cluster import KMeans                          #Clusteriza os dados não rotulados
from sklearn.preprocessing import StandardScaler            #Padroniza as caracteristicas dos dados
from sklearn.impute import SimpleImputer                    #Preenche valores ausentes em conjuntos de dados
from sklearn.preprocessing import MaxAbsScaler              #Dimensiona cada recurso para o intervalo [-1, 1]
from sklearn.neighbors import NearestNeighbors              #Implementa a aprendizagem de vizinhos mais próximos

import re       #Fornece operações com expressões regulares

# Leitura e manipulação do banco de dados

Dataset retirado do kaggle: https://www.kaggle.com/datasets/mittvin/anime-dataset

## Leitura do banco de dados

In [95]:
#Definindo os nomes das colunas do meu dataframe
columns = ['id', 'nome', 'genero', 'tipo', 'episodios', 'nota', 'contagem_de_votos']

#Lendo o arquivo
df = pd.read_csv('C:/Users/Acer/Desktop/Cursos/Semcon/Machine Learning/Projeto Final/anime.csv',
                sep = ',', names = columns, skiprows=1)

## Análise embasada no número de votos

In [96]:
media_contagem_votos = df['contagem_de_votos'].mean()           #Calcula a média da contagem de votos
mediana_contagem_votos = df['contagem_de_votos'].median()       #Calcula a mediana da contagem de votos

#Criando os subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=['Boxplot', 'Histograma'])
#Criando o gráfico de caixa
fig.add_trace(go.Box(y=df['contagem_de_votos'], name=' ',
                     marker=dict(color='blue')), row=1, col=1)
fig.add_shape(type='line', x0=-1, x1=1, y0=media_contagem_votos, y1=media_contagem_votos,
              line=dict(color='red', width=2), row=1, col=1)
fig.add_shape(type='line', x0=-1, x1=1, y0=mediana_contagem_votos, y1=mediana_contagem_votos,
              line=dict(color='green', width=2), row=1, col=1)
#Criando o Histograma
fig.add_trace(go.Histogram(x=df['contagem_de_votos'], marker=dict(color='orange'),
                           nbinsx=20), row=1, col=2)
fig.add_shape(type='line', x0=media_contagem_votos, x1=media_contagem_votos, y0=0, y1=1,
              line=dict(color='red', width=2), row=1, col=2)
fig.add_shape(type='line', x0=mediana_contagem_votos, x1=mediana_contagem_votos, y0=0, y1=1,
              line=dict(color='green', width=2), row=1, col=2)

#Renomeando os eixos
fig.update_xaxes(title_text='Quantidade de votos', row=1, col=1)
fig.update_yaxes(title_text='Divisão da quantidade de votos', row=1, col=1)
fig.update_xaxes(title_text='Quantidade de votos', row=1, col=2)
fig.update_yaxes(title_text='Frequência', row=1, col=2)

#Configuração da figura
fig.update_layout(title_text='Distribuição da Quantidade de Votos dos Animes', template='plotly_dark', showlegend=False)
fig.add_shape(type='line',
              x0=-1, x1=1, y0=media_contagem_votos, y1=media_contagem_votos,
              line=dict(color='red', width=2))
fig.add_shape(type='line',
              x0=-1, x1=1, y0=mediana_contagem_votos, y1=mediana_contagem_votos,
              line=dict(color='green', width=2))
fig.add_annotation(x=0.5, y=media_contagem_votos, text=f'Média: {media_contagem_votos:.2f}',
                   showarrow=True, arrowhead=5, ax=0, ay=-40)
fig.add_annotation(x=-0.5, y=mediana_contagem_votos, text=f'Mediana: {mediana_contagem_votos:.2f}',
                   showarrow=True, arrowhead=5, ax=0, ay=-40)

# Exibe o gráfico
fig.show()

## Análise embasada nas notas

In [97]:
media_notas = df['nota'].mean()     #Calcula a média da contagem de votos

#Cria um subplot com uma linha e duas colunas
fig = make_subplots(rows=1, cols=2, subplot_titles=['Histograma', 'Boxplot'])

#Adiciona o histograma
fig.add_trace(go.Histogram(x=df['nota'], nbinsx=20, marker=dict(color='orange')),
              row=1, col=1)

#Adiciona o boxplot
fig.add_trace(go.Box(y=df['nota'], name=' ', marker=dict(color='blue')),
              row=1, col=2)

#Adiciona uma linha representando a média no histograma
fig.add_shape(type='line', y0=0, y1=3000, x0=media_notas, x1=media_notas,
              line=dict(color='red', width=2), row=1, col=1)

#Adiciona uma anotação para a média no histograma
fig.add_annotation(x=media_notas, y=2900, text=f'Média: {media_notas:.2f}',
                   showarrow=True, arrowhead=5, ax=80, ay=0, row=1, col=1)

#Atualiza os eixos
fig.update_xaxes(title_text='Nota do Anime', row=1, col=1)
fig.update_yaxes(title_text='Contagem das notas', row=1, col=1)
fig.update_xaxes(title_text='Nota do Anime', row=1, col=2)
fig.update_yaxes(title_text='Nota do Anime', row=1, col=2)

#Atualiza o layout
fig.update_layout(title_text='Distribuição das notas', template='plotly_dark', height=400, width=900)

#Exibe o gráfico
fig.show()

## Consideração de filtro

In [127]:
df_filtrado = df[(df['contagem_de_votos']>=1000)&(df['nota']>=5)]       #Filtra os animes com nota maior ou igual a 5 e com 1000 ou mais participações

df_filtrado.loc[(df_filtrado["genero"]=="Hentai") & (df_filtrado["episodios"]=="Unknown"),"episodios"] = "1"       #Muda os dados Hentai para 1
df_filtrado.loc[(df_filtrado["tipo"]=="OVA") & (df_filtrado["episodios"]=="Unknown"),"episodes"] = "1"             #Muda os dados OVA para 1
df_filtrado.loc[(df_filtrado["tipo"] == "Movie") & (df_filtrado["episodios"] == "Unknown")] = "1"                  #Muda os dados desconhecidos para 1

#Informação de fintragem
print(f'O dataframe filtrado possui {len(df_filtrado)} linhas, já o original possui {len(df)}.\n{len(df) - len(df_filtrado)} apresentam menos de 1000 votos ou uma nota inferior a 5.')

O dataframe filtrado possui 6724 linhas, já o original possui 12294.
5570 apresentam menos de 1000 votos ou uma nota inferior a 5.


C:\Users\Acer\AppData\Local\Temp\ipykernel_456\397249625.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\397249625.py:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\397249625.py:5: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



# Análise Gráfica

## TOP 10 dos animes

### Geral

In [99]:
# Converte a coluna 'nota' para o tipo float
df_filtrado['nota'] = pd.to_numeric(df_filtrado['nota'], errors='coerce')

# Ordena os top 10 animes por nota em ordem decrescente
top_10_animes = df_filtrado.sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'})

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(title_text='Top 10 animes com maiores notas', template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\93642142.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Filme

In [100]:
top_10_animes_filmes = df_filtrado[df_filtrado['tipo'] == 'Movie'].sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes_filmes, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'},
             title='Filmes com Maiores Notas')

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

### TV

In [101]:
top_10_animes_TV = df_filtrado[df_filtrado['tipo'] == 'TV'].sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes_TV, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'},
             title='Seriados de TV com Maiores Notas')

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

### OVA (Original Video Animation)

In [102]:
top_10_animes_OVA = df_filtrado[df_filtrado['tipo'] == 'OVA'].sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes_OVA, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'},
             title='OVAs com Maiores Notas')

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

### Especial

In [103]:
top_10_animes_Especiais = df_filtrado[df_filtrado['tipo'] == 'Special'].sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes_Especiais, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'},
             title='Especiais com Maiores Notas')

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

### Musical

In [104]:
top_10_animes_Musicais = df_filtrado[df_filtrado['tipo'] == 'Music'].sort_values('nota', ascending=False).head(10)

# Cria o gráfico de barras horizontais classificado
fig = px.bar(top_10_animes_Musicais, x='nota', y='nome', orientation='h', text='nota',
             labels={'nota': 'Nota', 'nome': 'Nome do Anime'},
             title='Musicais com Maiores Notas')

# Inverte a ordem para ter a barra mais alta no topo
fig.update_layout(yaxis=dict(autorange='reversed'))
fig.update_layout(template='plotly_dark', height=400, width=900)

# Exibe o gráfico
fig.show()

# Machine Learning (Clusterização)

In [106]:
#Mapeando e limpando os Unknown
df_filtrado["episodios"] = df_filtrado["episodios"].map(lambda x:np.nan if x=="Unknown" else x)
df_filtrado["episodios"].fillna(df_filtrado["episodios"].median(),inplace = True)

#Removendo as notas inválidas
df_filtrado["nota"] = df_filtrado["nota"].astype(float)
df_filtrado["nota"].fillna(df_filtrado["nota"].median(),inplace = True)

#Criando dummies para os tipos dos animes
pd.get_dummies(df_filtrado[["tipo"]]).head()

#Convertendo os valores da contagem de votos para float
df_filtrado["contagem_de_votos"] = df_filtrado["contagem_de_votos"].astype(float)

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\1508898355.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\1508898355.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\1508898355.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

### Caracterização dos dados

In [107]:
#Caracterizando os animes
anime_features = pd.concat([df_filtrado["genero"].str.get_dummies(sep=","),pd.get_dummies(df_filtrado[["tipo"]]),df_filtrado[["nota"]],df_filtrado[["contagem_de_votos"]],df_filtrado["episodios"]],axis=1)
#Removendo caracteres especiais dos nomes dos animes
df_filtrado["nome"] = df_filtrado["nome"].map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))
anime_features.head()

C:\Users\Acer\AppData\Local\Temp\ipykernel_456\2027971363.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,...,tipo_1,tipo_Movie,tipo_Music,tipo_ONA,tipo_OVA,tipo_Special,tipo_TV,nota,contagem_de_votos,episodios
0,0,0,0,0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,9.37,200630.0,1
1,1,0,0,0,0,1,0,1,0,0,...,False,False,False,False,False,False,True,9.26,793665.0,64
2,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,9.25,114262.0,51
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,9.17,673572.0,24
4,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,9.16,151266.0,51


### Normalização

In [108]:
max_abs_scaler = MaxAbsScaler()                                     #Dimensiona cada recurso para o intervalo [-1, 1]
anime_features = max_abs_scaler.fit_transform(anime_features)       #Ajusta o scaler com base nos dados e realiza a reescalação dos valores para o intervalo [-1, 1].

### KNN

In [109]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(anime_features)       #Configurando para os 6 vizinhos mais próximos
distances, indices = nbrs.kneighbors(anime_features)                                    #Usa o modelo treinado para encontrar os vizinhos

### Funções

#### Busca o indice do anime

In [110]:
def indexador(name):
    return df[df["nome"]==name].index.tolist()[0]

indexador("Hunter x Hunter")  

112

#### Busca títulos de animes com base em uma parte do nome

In [111]:
titulos = list(df.nome.values)
def busca_anime_parcial(partial):
    for name in titulos:
        if partial in name:
            print(name,titulos.index(name))

#### Busca os vizinhos próximos para o anime informado

In [112]:
def recomendador(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(df.iloc[id]["nome"])
    if query:
        found_id = indexador(query)
        for id in indices[found_id][1:]:
            print(df.iloc[id]["nome"])

# Testes

In [113]:
busca_anime_parcial("JoJo")

JoJo no Kimyou na Bouken: Stardust Crusaders 2nd Season 64
JoJo no Kimyou na Bouken: Diamond wa Kudakenai 76
JoJo no Kimyou na Bouken (TV) 95
JoJo no Kimyou na Bouken: Stardust Crusaders 257
JoJo no Kimyou na Bouken: Phantom Blood 914
JoJo no Kimyou na Bouken 1668
JoJo no Kimyou na Bouken: Adventure 2581


In [114]:
recomendador("JoJo no Kimyou na Bouken")

Black Bullet
Garo: Honoo no Kokuin
Kore wa Zombie Desu ka? OVA
Jungle wa Itsumo Hare nochi Guu Final
Ichigo Mashimaro Encore


In [115]:
recomendador("Hunter x Hunter")

Hunter x Hunter (2011)
Fate/stay night: Unlimited Blade Works 2nd Season - Sunny Day
Saint Seiya: Shinku no Shounen Densetsu
Naruto: The Cross Roads
Mahou no Princess Minky Momo


In [116]:
recomendador("One Punch Man")

One Punch Man: Road to Hero
One Punch Man Specials
Urusei Yatsura Movie 3: Remember My Love
Bakusou Kyoudai Let&#039;s &amp; Go MAX
Candy☆Boy: Side Story For Archive


In [117]:
recomendador("One Piece")

Be-Bop Highschool
Dragon Ball Z
Shingeki no Kyojin
Dragon Ball Kai
Saiunkoku Monogatari Soushuuhen


In [118]:
recomendador("Death Note")

Higurashi no Naku Koro ni Kai
Death Note Rewrite
Jigoku Shoujo Mitsuganae
Bokura no Live Kimi to no Life
Ayakashi


In [119]:
recomendador("Fullmetal Alchemist: Brotherhood")

Fullmetal Alchemist
Magi: The Labyrinth of Magic
Magi: The Kingdom of Magic
Densetsu no Yuusha no Densetsu
Magi: Sinbad no Bouken (TV)


In [120]:
recomendador("Pokemon")

Pumpkin Scissors
Supernatural The Animation
Pita Ten
Sangokushi (1985)
Fire Tripper


In [121]:
recomendador("Mob Psycho 100")

Scryed Alteration II: Quan
Space Pirate Captain Herlock: Outside Legend - The Endless Odyssey
Monster Farm: Legend e no Michi
Shamanic Princess
Full Metal Panic? Fumoffu


In [122]:
recomendador("Gintama")

Gintama&#039;
Gintama°
Gintama&#039;: Enchousen
Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare
Gintama Movie: Shinyaku Benizakura-hen


In [123]:
busca_anime_parcial("Haikyuu")

Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou 5
Haikyuu!! Second Season 14
Haikyuu!! 43
Haikyuu!! Movie 1: Owari to Hajimari 625
Haikyuu!! OVA 626
Haikyuu!! Movie 2: Shousha to Haisha 851
Haikyuu!! Quest Picture Drama 4599


In [124]:
recomendador("Haikyuu!!")

Haikyuu!! Second Season
Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou
Slam Dunk
Kuroko no Basket 2nd Season
Kuroko no Basket
